In [ ]:
# #final code start

In [ ]:
import random
from collections import defaultdict
import numpy as np

# ====================== INPUT DATA ======================
years = {
    '1st': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['Math', 'Physics', 'Chemistry', 'English', 'CS', 'Electronics'],
        'professors': {
            'Math': ['Prof_M1', 'Prof_M2'],
            'Physics': ['Prof_P1', 'Prof_P2'],
            'Chemistry': ['Prof_C1', 'Prof_C2'],
            'English': ['Prof_E1', 'Prof_E2'],
            'CS': ['Prof_CS1', 'Prof_CS2'],
            'Electronics': ['Prof_EL1', 'Prof_EL2']
        }
    },
    '2nd': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['DS', 'OS', 'DBMS', 'OOP', 'DE', 'CA'],
        'professors': {
            'DS': ['Prof_PRO', 'Prof_VU'],
            'OS': ['Prof_SP', 'Prof_PC'],
            'DBMS': ['Prof_AN', 'Prof_GR'],
            'OOP': ['Prof_SP', 'Prof_VC'],
            'DE': ['Prof_KL', 'Prof_SD'],
            'CA': ['Prof_BS', 'Prof_PK','Prof_AA']
        }
    },
    '3rd': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['ML', 'NLC', 'CN', 'CVDL', 'DAA', 'CC'],
        'professors': {
            'ML': ['Prof_PK', 'Prof_SHM'],
            'NLC': ['Prof_PBT', 'Prof_TV'],
            'CN': ['Prof_SG', 'Prof_GR'],
            'CVDL': ['Prof_SJ', 'Prof_RK'],
            'DAA': ['Prof_AV', 'Prof_JP'],
            'CC': ['Prof_JBB', 'Prof_BS']
        }
    }
}

classrooms = ['N301', 'N302', 'N303', 'N304', 'N305', 'N306', 'N307', 'N308','N201', 'N202', 'N203', 'N204', 'N205', 'N206', 'N207', 'N208']
time_slots = ['9-10', '10-11', '11-12', '12-1', '2-3', '3-4']  # 6 hours/day
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']  # 5 days

# Constants
SUBJECT_HOURS_REQUIRED = 4  # Each subject taught 4 hours per division weekly

class TimetableGA:
    def __init__(self, population_size=300, generations=400, crossover_rate=0.9, mutation_rate=0.12):
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.best_chromosome = None
        self.best_fitness = -float('inf')

        # Collect all professors across all years
        self.all_professors = list(set(
            p for year in years.values()
            for sublist in year['professors'].values()
            for p in sublist
        ))

        # Collect all divisions with year prefix (e.g., '1st_A')
        self.all_divisions = [
            f"{year}_{div}"
            for year, data in years.items()
            for div in data['divisions']
        ]

        # Calculate total hours required per division
        self.HOURS_PER_DIVISION = {
            year: len(data['subjects']) * SUBJECT_HOURS_REQUIRED
            for year, data in years.items()
        }
    # ====================== INITIALIZATION ======================
    def initialize_population(self):
        #Create initial population of valid schedules
        self.population = []
        for _ in range(self.population_size):
            chromosome = self.create_valid_chromosome()
            self.population.append(chromosome)

    # ====================== CHROMOSOME ENCODING ======================
    def create_valid_chromosome(self):
        # 3D structure: [days][time_slots][classrooms]  holds lecture details (division, subject, professor) initially NONE
        schedule = [[[None for _ in classrooms] for _ in time_slots] for _ in days]

        # Track subject hours per division
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # Track total hours per division
        division_total_hours = {
            f"{year}_{div}": 0
            for year in years
            for div in years[year]['divisions']
        }

        # Assign classes until all requirements are met
        while not self.all_requirements_met(division_subject_hours, division_total_hours):
            # Select a division and subject that needs hours
            div, subj = self.select_remaining_assignment(division_subject_hours)
            if div is None:
                break

            year = div.split('_')[0]

            # Find a valid timeslot
            day_idx, slot_idx, room_idx = self.find_valid_timeslot(
                schedule, div, subj, year
            )
            if day_idx is None:
                break  # Couldn't find valid slot, will repair later

            # Assign to schedule
            prof = random.choice(years[year]['professors'][subj])
            schedule[day_idx][slot_idx][room_idx] = {
                'division': div,
                'subject': subj,
                'prof': prof,
                'year': year
            }

            # Update tracking
            division_subject_hours[div][subj] += 1
            division_total_hours[div] += 1

        return self.repair_schedule(schedule)

    # ====================== FITNESS FUNCTION ======================
    # This function evaluates the fitness of a given timetable(chromosome). and the fitness score determines how good a timetable is based on various constraints.
    # Key Idea:
    # Lower penalty = Better timetable
    # The function tracks violations (e.g., incorrect hours, professor overload) and assigns penalty points.
    # A negative penalty is returned, cause our penulty system works in reverse( lower values are better), so -panulty converts this to a maximization problem.

    def calculate_fitness(self, chromosome):
        penalty = 0 # keeps track of rule violations.

        # Tracking dictionaries
        # 1. Tracking how many hours each subject has been assigned per division.
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # {
        #   '1st_A': {'Math': 0, 'Physics': 0, ...},
        #   '2nd_B': {'OS': 0, 'DBMS': 0, ...},
        # }


        # 2. Tracking total teaching hours per division.
        division_total_hours = {
            f"{year}_{div}": 0
            for year in years
            for div in years[year]['divisions']
        }


        # {
        #   '1st_A': 0,
        #   '2nd_B': 0,
        # }


        # 3. Tracking Professors Workload
        # Tracks how many hours each professor teaches daily and weekly.
        prof_daily_hours = {prof: {day: 0 for day in days} for prof in self.all_professors}
        prof_total_hours = {prof: 0 for prof in self.all_professors}


        # prof_daily_hours
        # {
        #   'Prof_M1': {'Monday': 0, 'Tuesday': 0, ...}, #total hours on perticular day
        #   'Prof_VC': {'Monday': 0, 'Tuesday': 0, ...},
        # }

        # prof_total_hours
        # {
        #   'Prof_M1': 0,  # Total teaching hours in the week
        #   'Prof_VC': 0,
        # }

        # 4. Tracking Room Usage
        # Tracks how many times each classroom is used
        room_utilization = {room: 0 for room in classrooms}

        # {
        #   'N301': 0,  # Used 0 times in a week initially.
        #   'N302': 0,
        # }

        # 5. Tracking Section Workload
        # Tracks how many hours a section has classes per day.
        section_daily_hours = {div: {day: 0 for day in days} for div in self.all_divisions}

        # section_dauky_hours
        # {
        #   '1st_A': {'Monday': 0, 'Tuesday': 0, ...},
        #   '2nd_B': {'Monday': 0, 'Tuesday': 0, ...},
        # }

        # 6. Tracks consecutive hours of teaching to avoid overloading students.
        section_consecutive_hours = {div: {day: [] for day in days} for div in self.all_divisions}

        # 1. Processing the chromosome: (5x6x16) (days x slots x rooms)
        for day_idx, day in enumerate(chromosome):
            for slot_idx, slot in enumerate(day):
                for room_idx, assignment in enumerate(slot):
                    if assignment:
                        div = assignment['division']
                        subj = assignment['subject']
                        prof = assignment['prof']
                        year = assignment['year']


                        # Assignment (Monday, 9-10 AM, N301):
                        # {
                        #   'division': '1st_A',
                        #   'subject': 'Math',
                        #   'prof': 'Prof_M1',
                        #   'year': '1st'
                        # }


                        # Track subject hours
                        division_subject_hours[div][subj] += 1 # increasing the count for that div with perticular subject
                        division_total_hours[div] += 1 # and total hours for division also increased.

                        # Track professor hours
                        prof_daily_hours[prof][days[day_idx]] += 1 # increasing professor working hour on perticular day.
                        prof_total_hours[prof] += 1 # increasing professor's total hours for week.

                        # Track room utilization
                        room_utilization[classrooms[room_idx]] += 1 # room utilization is increased by 1.

                        # Track section daily hours
                        # Tracks daily teaching load and consecutive teaching hours for each division.
                        section_daily_hours[div][days[day_idx]] += 1 # for perticular division what is the day workload.
                        section_consecutive_hours[div][days[day_idx]].append(slot_idx)


        # 2. Calculate penalties for each constraint
        #------------------------------ J
        # a) Each subject must be taught exactly 4 hours per division
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] != SUBJECT_HOURS_REQUIRED: # for perticular division and for perticular subject if subject hours not match to required subject hours(3) then it is not good solution.
                        penalty += 10000 * abs(division_subject_hours[full_div][subj] - SUBJECT_HOURS_REQUIRED) # variable penalty, if diff is large then penalty will large.

        # b) Each division must have correct total hours (24) cause 6 subjects and 4 hours in week
        for year in years:
            expected_hours = self.HOURS_PER_DIVISION[year]
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                if division_total_hours[full_div] != expected_hours: # for all division's total hours we re checking whether it is equal to expected hours if not large penalty.
                    penalty += 10000 * abs(division_total_hours[full_div] - expected_hours)

        #---------------------------- M
        # c) Professor daily workload balance
        # Goal: Ensure professors have a balanced workload across all days (no day is too overloaded or underloaded)

        # average daily hours per professor
        avg_prof_daily = sum(prof_total_hours.values()) / (len(self.all_professors) * len(days))
        # total teaching hours across all professors / average hours per professor per day
        for prof in self.all_professors: # for each professor
            for day in days: #on each day
                # compute the deviation from this average
                deviation = abs(prof_daily_hours[prof][day] - avg_prof_daily)
                # hours taught by the professor on that day - average daily hours per professor
                penalty += 1000 * deviation



        # d) Avoid too many consecutive hours for professors
        # Prevent professors from teaching back-to-back for too long

        # INITIALIZE dictionary to track professor teaching hours
        prof_consecutive_hours = {} # empty dictionary
        for prof in self.all_professors: # for every professor
            prof_consecutive_hours[prof] = {} # sub-dictionary for each professor - store daily schedules
            for day in days: # for each day
                # initialize empty list to store time slots when professor teaches
                prof_consecutive_hours[prof][day] = []

        # COLLECT all time slots where each professor teaches
        for day_idx in range(len(chromosome)): #for each day
            day = chromosome[day_idx] #get schedule for current day
            for slot_idx in range(len(day)): #for each time slot in the day
                slot = day[slot_idx] #get all classroom assignments for current time slot
                for room_idx in range(len(slot)): #for each classroom in time slot
                    assignment = slot[room_idx] #get the assignment (class) scheduled in this classroom-time slot
                    if assignment is not None: #if classroom not empty
                        prof = assignment['prof'] #get professor teaching this class
                        day_name = days[day_idx] #get name of current day
                        prof_consecutive_hours[prof][day_name].append(slot_idx) #record the time slot index when the professor teaches on this day

        # check for CONSECUTIVE teaching hours
        for prof in self.all_professors: #loops through all professors again to analyze their schedules
            for day in days: # for each professor, checks each day's schedule
                slots = prof_consecutive_hours[prof][day] # get all time slots when the professor teaches on this day
                slots = sorted(slots) #sort chronologically
                consecutive_count = 1
                for i in range(1, len(slots)): #loops through the sorted time slots (starting from second)
                    if slots[i] - slots[i-1] == 1: # consecutive hours (if current slot is immediately after previous slot )
                        consecutive_count += 1
                        if consecutive_count > 2:  # prof teaches more than 2 consecutive hours
                            penalty += 1000
                    else:
                        consecutive_count = 1

        # e) Idle hours should be at end of day
        # Goal: Prefer schedules where empty slots are at the end of the day rather than between classes
        for day_idx, day in enumerate(chromosome): #for each day in the timetable(chromosome)
            last_used_slot = -1 # track the last used time slot (last_used_slot)
            for slot_idx in range(len(time_slots)): # for each time slot in the day
                # check whether any classroom in a given time slot is occupied
                slot_used = any(assignment is not None for assignment in day[slot_idx]) #at least one class is in that time slot
                #                                    class-assignment in class wise list of assignments in specific time slot

                if slot_used:
                    last_used_slot = slot_idx # if the slot is used, update last_used_slot to the current slot index
                elif last_used_slot != -1:  # if found idle slot after used slot, give penalty
                    penalty += 50 * (slot_idx - last_used_slot)
                    # farther the idle slot is from the last used slot, higher the penalty

        return -penalty  # return negative penalty (higher is better)
        # for us, higher penalty = worse timetable, lower penalty = better timetable
        # GA maximizes fitness functions so for GA, higher is better, which is not the case for us
        # hence return -penalty

    # # ====================== INITIALIZATION ======================
    # def initialize_population(self):
    #     #Create initial population of valid schedules
    #     self.population = []
    #     for _ in range(self.population_size):
    #         chromosome = self.create_valid_chromosome()
    #         self.population.append(chromosome)

    # ====================== SELECTION OPERATOR ======================
    #Tournament selection with elitism
    #takes fitness scores of the current population as input
    def selection(self, fitness_scores):
        selected = [] #list of chromosomes (timetables) that will be used for reproduction (crossover and mutation)
        # keep top 10% elites
        elite_size = max(1, int(self.population_size * 0.1)) #10% of the population size, min 1
        elite_indices = np.argsort(fitness_scores)[-elite_size:] # get indices of the top 10% fittest individuals
        selected.extend([self.population[i] for i in elite_indices]) #adds these elites directly to selected list for next generation

        # tournament selection for rest 90%
        for _ in range(self.population_size - elite_size): #90%
            candidates = random.sample(range(self.population_size), 5) # randomly select 5 candidates from the population
            best = max(candidates, key=lambda x: fitness_scores[x]) #select BEST individual with the highest fitness score
            selected.append(self.population[best]) # add the best individual to the selected list

        return selected #list of chromosomes (timetables) that will be used for reproduction (crossover and mutation)

    # ====================== REPRODUCTION OPERATORS ======================
    def crossover(self, parent1, parent2):
        #Enhanced crossover that preserves valid structures
        if random.random() > self.crossover_rate:
            return [row[:] for row in parent1], [row[:] for row in parent2]

        # Create empty children
        child1 = [[[None for _ in classrooms] for _ in time_slots] for _ in days]
        child2 = [[[None for _ in classrooms] for _ in time_slots] for _ in days]

        # Perform uniform crossover for each day
        for day_idx in range(len(days)):
            if random.random() < 0.5:  # Swap entire day
                child1[day_idx] = [slot[:] for slot in parent1[day_idx]]
                child2[day_idx] = [slot[:] for slot in parent2[day_idx]]
            else:
                child1[day_idx] = [slot[:] for slot in parent2[day_idx]]
                child2[day_idx] = [slot[:] for slot in parent1[day_idx]]

        return self.repair_schedule(child1), self.repair_schedule(child2)

    def mutate(self, chromosome):
        #mutation that maintains validity
        for _ in range(5):  # Perform 5 mutations
            # Find two random assignments to swap
            day1, day2 = random.sample(range(len(days)), 2)
            slot1, slot2 = random.sample(range(len(time_slots)), 2)
            room1, room2 = random.sample(range(len(classrooms)), 2)

            # Perform swap if both assignments exist
            if (chromosome[day1][slot1][room1] and
                chromosome[day2][slot2][room2]):
                # Check if swap would cause conflicts
                temp1 = chromosome[day1][slot1][room1].copy()
                temp2 = chromosome[day2][slot2][room2].copy()

                # Simulate swap
                chromosome[day1][slot1][room1] = None
                chromosome[day2][slot2][room2] = None

                # Check for conflicts
                conflict1 = self.has_conflict(chromosome, day1, slot1, temp2)
                conflict2 = self.has_conflict(chromosome, day2, slot2, temp1)

                if not conflict1 and not conflict2:
                    chromosome[day1][slot1][room1] = temp2
                    chromosome[day2][slot2][room2] = temp1
                else:
                    # Revert if conflict
                    chromosome[day1][slot1][room1] = temp1
                    chromosome[day2][slot2][room2] = temp2

        return self.repair_schedule(chromosome)

    # ====================== HELPER METHODS ======================
    def all_requirements_met(self, division_subject_hours, division_total_hours):
        for year in years:
            expected_hours = self.HOURS_PER_DIVISION[year]
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                # Check subject hours
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                        return False
                # Check total hours
                if division_total_hours[full_div] < expected_hours:
                    return False
        return True

    def select_remaining_assignment(self, division_subject_hours):
        remaining = []
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                        remaining.append((full_div, subj))
        return random.choice(remaining) if remaining else (None, None)

    def find_valid_timeslot(self, schedule, div, subj, year):
        # Try random slots first for diversity
        for _ in range(100):
            day_idx = random.randint(0, len(days)-1)
            slot_idx = random.randint(0, len(time_slots)-1)
            room_idx = random.randint(0, len(classrooms)-1)




             #check if room is empty --> if div is free
            if schedule[day_idx][slot_idx][room_idx] is None:
                # Check if division is available
                div_busy = any(
                    assignment and assignment['division'] == div
                    for assignment in schedule[day_idx][slot_idx]
                )
                if not div_busy:
                    # Check if professor is available
                    prof = random.choice(years[year]['professors'][subj])
                    prof_busy = any(
                        assignment and assignment['prof'] == prof
                        for assignment in schedule[day_idx][slot_idx]
                    )
                    if not prof_busy:
                        return day_idx, slot_idx, room_idx

        # If random search fails, do exhaustive search
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    if schedule[day_idx][slot_idx][room_idx] is None:
                        div_busy = any(
                            assignment and assignment['division'] == div
                            for assignment in schedule[day_idx][slot_idx]
                        )
                        if not div_busy:
                            prof = random.choice(years[year]['professors'][subj])
                            prof_busy = any(
                                assignment and assignment['prof'] == prof
                                for assignment in schedule[day_idx][slot_idx]
                            )
                            if not prof_busy:
                                return day_idx, slot_idx, room_idx
        return None, None, None

    def has_conflict(self, chromosome, day_idx, slot_idx, assignment):
        if not assignment:
            return False

        # Check division conflict
        for room_idx in range(len(classrooms)):
            existing = chromosome[day_idx][slot_idx][room_idx]
            if existing and existing['division'] == assignment['division']:
                return True

        # Check professor conflict
        for room_idx in range(len(classrooms)):
            existing = chromosome[day_idx][slot_idx][room_idx]
            if existing and existing['prof'] == assignment['prof']:
                return True

        return False

    def repair_schedule(self, chromosome):
        # Creates a dictionary to track how many hours each subject has been assigned per division.
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # First pass: count current assignments Iterates through the timetable (chromosome) and counts how many times each subject has been assigned for every division.
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    assignment = chromosome[day_idx][slot_idx][room_idx]
                    if assignment:
                        div = assignment['division']
                        subj = assignment['subject']
                        division_subject_hours[div][subj] += 1

        # Second pass: add missing assignments
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    if chromosome[day_idx][slot_idx][room_idx] is None:#loops through day,slot,room if it is empty then it is capable of getting any assignment which does not have any conflict

                          # Iterates over all divisions and subjects to find one that still needs more hours
                        for year in years:
                            for div in years[year]['divisions']:
                                full_div = f"{year}_{div}"
                                for subj in years[year]['subjects']:
                                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                                        # Check if this slot is available
                                        test_assignment = {
                                            'division': full_div,
                                            'subject': subj,
                                            'prof': years[year]['professors'][subj][0],
                                            'year': year
                                        }
                                        if not self.has_conflict(chromosome, day_idx, slot_idx, test_assignment):
                                            prof = random.choice(years[year]['professors'][subj])
                                            chromosome[day_idx][slot_idx][room_idx] = {
                                                'division': full_div,
                                                'subject': subj,
                                                'prof': prof,
                                                'year': year
                                            }
                                            division_subject_hours[full_div][subj] += 1
                                            break
                                else:
                                    continue
                                break

        return chromosome

    # ====================== MAIN ALGORITHM ======================
    # ---------------J
    # This method runs the genetic algorithm (GA) to generate an optimal class timetable.
    # It follows the standard GA workflow:
    # 1. Initialize Population
    # 2. Evaluate Fitness
    # 3. Selection
    # 4. Crossover & Mutation
    # 5. Repeat for multiple generations
    # 6. Return the best timetable (chromosome)

    def run(self):
        # Creating an initial population (random valid timetables).
        # Each timetable (chromosome) is a 3D list representing [days][time_slots][rooms]
        self.initialize_population()

        # GA runs for a fixed number of generations (200)
        # Each generation evolves the timetable toward an optimal solution.
        for generation in range(self.generations):

            # Calculating Fitness for Each Chromosome (higher is better).
            # fitness_scores → Stores fitness for all schedules in the population.
            fitness_scores = [self.calculate_fitness(chromosome) for chromosome in self.population]

            # max_fitness is maximum fitness among all 100 individual's fitness for perticular generation.
            max_fitness = max(fitness_scores)

            # if max_fitness is greater than the best_fitness which is good then replace best_fitness with max_fitness.
            if max_fitness > self.best_fitness:
                self.best_fitness = max_fitness
                best_idx = fitness_scores.index(max_fitness) # maintaining the best_index.

                # Storing the best timetable found so far.
                self.best_chromosome = [
                    [
                        [
                            {k: v for k, v in assignment.items()} if assignment else None
                            for assignment in slot
                        ]
                        for slot in day
                    ]
                    for day in self.population[best_idx]
                ]

            # Print progress : Every 10 generations, prints: Best fitness, Average fitness of the population
            if generation % 10 == 0:
                avg_fitness = sum(fitness_scores) / len(fitness_scores)
                print(f"Gen {generation}: Best {self.best_fitness:.1f}, Avg {avg_fitness:.1f}")

            # Selection (Choosing Parents)
            selected = self.selection(fitness_scores) # Higher fitness increases selection probability.

            # Crossover and mutation
            new_population = []
            for i in range(0, len(selected), 2):
                parent1 = selected[i]
                parent2 = selected[i+1] if i+1 < len(selected) else selected[i]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.extend([self.mutate(child1), self.mutate(child2)])

            # Ensure population size
            new_population = new_population[:self.population_size]

            # Replace the worst individuals in new population with best from old population
            if self.best_chromosome:
                # Find worst in new population
                new_fitness = [self.calculate_fitness(chromosome) for chromosome in new_population]
                worst_idx = np.argmin(new_fitness)

                # Replace worst with best from previous generation
                new_population[worst_idx] = [
                    [
                        [
                            {k: v for k, v in assignment.items()} if assignment else None
                            for assignment in slot
                        ]
                        for slot in day
                    ]
                    for day in self.best_chromosome
                ]

            self.population = new_population

        return self.best_chromosome #The best timetable (least conflicts, most optimized) is returned.

    # ====================== OUTPUT METHODS ======================
    #------------J
    #  method is responsible for printing the generated timetable
    def print_schedule(self, chromosome):
        print("\n=== COMPLETE TIMETABLE (ALL DIVISIONS) ===") # Prints the full timetable for all divisions
        self.print_complete_schedule(chromosome)

        print("\n\n=== INDIVIDUAL DIVISION TIMETABLES ===") # Prints individual division-wise timetables
        self.print_division_schedules(chromosome)

        # Print statistics
        self.print_statistics(chromosome) # Prints statistics about the schedule.

    def print_complete_schedule(self, chromosome):

        for day_idx, day in enumerate(chromosome):
            print(f"\n{days[day_idx].upper()}") # Printing the day name (Monday, Tuesday, etc.) in uppercase.
            print(f"{'Time':<8}", end="") # Column for time slots

            #  Print room names as table headers
            for room in classrooms:
                print(f"{room:<30}", end="")
            print("\n" + "-"*180) # Table separator

            # Print time slots and class assignments
            for slot_idx in range(len(time_slots)):
                print(f"{time_slots[slot_idx]:<8}", end="") # Print time slot

                for room_idx in range(len(classrooms)):
                    assignment = day[slot_idx][room_idx] # Get class assignment for that time slot & room

                    if assignment and isinstance(assignment, dict): # Check if there is a valid class
                        output = (f"{assignment['year']}_{assignment['division']}-"
                                f"{assignment['subject'][:5]} ({assignment['prof']})")
                        print(f"{output:<30}", end="") # Print class info (year, division, subject, prof)
                    else:
                        print(f"{'---':<30}", end="") # Empty slot indicator
                print()

    def print_division_schedules(self, chromosome):
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                print(f"\n\n=== TIMETABLE FOR {full_div} ===")

                # Create a division-specific timetable
                div_timetable = [[None for _ in time_slots] for _ in days]

                # Fill the division timetable
                for day_idx, day in enumerate(chromosome):
                    for slot_idx, slot in enumerate(day):
                        for room_idx, assignment in enumerate(slot):
                            if assignment and assignment['division'] == full_div:
                                div_timetable[day_idx][slot_idx] = {
                                    'subject': assignment['subject'],
                                    'prof': assignment['prof'],
                                    'room': classrooms[room_idx]
                                }

                # Print the division timetable
                for day_idx, day in enumerate(div_timetable):
                    print(f"\n{days[day_idx].upper()}")
                    print(f"{'Time':<8}{'Subject':<15}{'Professor':<20}{'Room':<10}")
                    print("-"*50)

                    for slot_idx in range(len(time_slots)):
                        assignment = day[slot_idx]
                        if assignment:
                            print(f"{time_slots[slot_idx]:<8}"
                                  f"{assignment['subject']:<15}"
                                  f"{assignment['prof']:<20}"
                                  f"{assignment['room']:<10}")
                        else:
                            print(f"{time_slots[slot_idx]:<8}{'---':<15}{'---':<20}{'---':<10}")

    def print_statistics(self, chromosome):
        print("\n=== STATISTICS ===")

        # Professor workload
        print("\nProfessor Workload:")
        prof_hours = defaultdict(int)
        for day in chromosome:
            for slot in day:
                for assignment in slot:
                    if assignment and isinstance(assignment, dict):
                        prof_hours[assignment['prof']] += 1

        avg_hours = sum(prof_hours.values()) / len(prof_hours) if prof_hours else 0
        print(f"Average hours per professor: {avg_hours:.1f}")
        for prof, hours in sorted(prof_hours.items()):
            print(f"{prof}: {hours} hours (deviation: {abs(hours-avg_hours):.1f})")

        # Division statistics by year
        print("\nDivision Coverage by Year:")
        for year in years:
            print(f"\n{year} Year:")
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                subj_hours = defaultdict(int)
                for day in chromosome:
                    for slot in day:
                        for assignment in slot:
                            if (assignment and isinstance(assignment, dict) and
                                assignment['division'] == full_div):
                                subj_hours[assignment['subject']] += 1

                total = sum(subj_hours.values())
                print(f"{div}: {total} total hours")
                for subj in years[year]['subjects']:
                    print(f"  {subj}: {subj_hours.get(subj, 0)}/{SUBJECT_HOURS_REQUIRED} hours")

# ====================== RUN THE ALGORITHM ======================
if __name__ == "__main__":
    # Create and run the GA
    print("Generating optimal timetable for 3 years with 4 divisions each...")
    ga = TimetableGA(population_size=300, generations=400)
    best_schedule = ga.run()

    # Print results
    ga.print_schedule(best_schedule)

    # Print final fitness
    print(f"\nFinal Fitness Score: {ga.best_fitness}")

Generating optimal timetable for 3 years with 4 divisions each...
Gen 0: Best -145854.5, Avg -178556.6
Gen 10: Best -141363.6, Avg -444975.4
Gen 20: Best -130890.9, Avg -180534.5
Gen 30: Best -122890.9, Avg -144849.8
Gen 40: Best -115909.1, Avg -146985.6
Gen 50: Best -107945.5, Avg -131242.7
Gen 60: Best -100963.6, Avg -109186.0
Gen 70: Best -97490.9, Avg -103281.9
Gen 80: Best -93472.7, Avg -120216.1
Gen 90: Best -89981.8, Avg -105307.1
Gen 100: Best -85509.1, Avg -103962.7
Gen 110: Best -81018.2, Avg -90604.1
Gen 120: Best -79527.3, Avg -79882.0
Gen 130: Best -79018.2, Avg -82008.9
Gen 140: Best -78036.4, Avg -79775.5
Gen 150: Best -77527.3, Avg -80514.2
Gen 160: Best -76036.4, Avg -78055.5
Gen 170: Best -76036.4, Avg -76761.6
Gen 180: Best -76036.4, Avg -77014.0
Gen 190: Best -74545.5, Avg -75693.1
Gen 200: Best -74545.5, Avg -75215.4
Gen 210: Best -74545.5, Avg -74999.2
Gen 220: Best -74545.5, Avg -75732.3
Gen 230: Best -74545.5, Avg -75538.2
Gen 240: Best -74545.5, Avg -75038.7
Ge

In [ ]:
import random
from collections import defaultdict
import numpy as np

# ====================== INPUT DATA ======================
years = {
    '1st': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['Math', 'Physics', 'Chemistry', 'English', 'CS', 'Electronics'],
        'professors': {
            'Math': ['Prof_M1', 'Prof_M2'],
            'Physics': ['Prof_P1', 'Prof_P2'],
            'Chemistry': ['Prof_C1', 'Prof_C2'],
            'English': ['Prof_E1', 'Prof_E2'],
            'CS': ['Prof_CS1', 'Prof_CS2'],
            'Electronics': ['Prof_EL1', 'Prof_EL2']
        }
    },
    '2nd': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['DS', 'OS', 'DBMS', 'OOP', 'DE', 'CA'],
        'professors': {
            'DS': ['Prof_PRO', 'Prof_VU'],
            'OS': ['Prof_SP', 'Prof_PC'],
            'DBMS': ['Prof_AN', 'Prof_GR'],
            'OOP': ['Prof_SP', 'Prof_VC'],
            'DE': ['Prof_KL', 'Prof_SD'],
            'CA': ['Prof_BS', 'Prof_PK','Prof_AA']
        }
    },
    '3rd': {
        'divisions': ['A', 'B', 'C', 'D'],
        'subjects': ['ML', 'NLC', 'CN', 'CVDL', 'DAA', 'CC'],
        'professors': {
            'ML': ['Prof_PK', 'Prof_SHM'],
            'NLC': ['Prof_PBT', 'Prof_TV'],
            'CN': ['Prof_SG', 'Prof_GR'],
            'CVDL': ['Prof_SJ', 'Prof_RK'],
            'DAA': ['Prof_AV', 'Prof_JP'],
            'CC': ['Prof_JBB', 'Prof_BS']
        }
    }
}

classrooms = ['N301', 'N302', 'N303', 'N304', 'N305', 'N306', 'N307', 'N308','N201', 'N202', 'N203', 'N204', 'N205', 'N206', 'N207', 'N208']
time_slots = ['9-10', '10-11', '11-12', '12-1', '2-3', '3-4']  # 6 hours/day
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']  # 5 days

# Constants
SUBJECT_HOURS_REQUIRED = 4  # Each subject taught 4 hours per division weekly

class TimetableGA:
    def __init__(self, population_size=100, generations=200, crossover_rate=0.9, mutation_rate=0.12):
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.best_chromosome = None
        self.best_fitness = -float('inf')

        # Collect all professors across all years
        self.all_professors = list(set(
            p for year in years.values()
            for sublist in year['professors'].values()
            for p in sublist
        ))

        # Collect all divisions with year prefix (e.g., '1st_A')
        self.all_divisions = [
            f"{year}_{div}"
            for year, data in years.items()
            for div in data['divisions']
        ]

        # Calculate total hours required per division
        self.HOURS_PER_DIVISION = {
            year: len(data['subjects']) * SUBJECT_HOURS_REQUIRED
            for year, data in years.items()
        }

    # ====================== CHROMOSOME ENCODING ======================
    def create_valid_chromosome(self):
        # 3D structure: [days][time_slots][classrooms]  holds lecture details (division, subject, professor) initially NONE
        schedule = [[[None for _ in classrooms] for _ in time_slots] for _ in days]

        # Track subject hours per division
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # Track total hours per division
        division_total_hours = {
            f"{year}_{div}": 0
            for year in years
            for div in years[year]['divisions']
        }

        # Assign classes until all requirements are met
        while not self.all_requirements_met(division_subject_hours, division_total_hours):
            # Select a division and subject that needs hours
            div, subj = self.select_remaining_assignment(division_subject_hours)
            if div is None:
                break

            year = div.split('_')[0]

            # Find a valid timeslot
            day_idx, slot_idx, room_idx = self.find_valid_timeslot(
                schedule, div, subj, year
            )
            if day_idx is None:
                break  # Couldn't find valid slot, will repair later

            # Assign to schedule
            prof = random.choice(years[year]['professors'][subj])
            schedule[day_idx][slot_idx][room_idx] = {
                'division': div,
                'subject': subj,
                'prof': prof,
                'year': year
            }

            # Update tracking
            division_subject_hours[div][subj] += 1
            division_total_hours[div] += 1

        return self.repair_schedule(schedule)

    # ====================== FITNESS FUNCTION ======================
    # This function evaluates the fitness of a given timetable(chromosome). and the fitness score determines how good a timetable is based on various constraints.
    # Key Idea:
    # Lower penalty = Better timetable
    # The function tracks violations (e.g., incorrect hours, professor overload) and assigns penalty points.
    # A negative penalty is returned, cause our penulty system works in reverse( lower values are better), so -panulty converts this to a maximization problem.

    def calculate_fitness(self, chromosome):
        penalty = 0 # keeps track of rule violations.

        # Tracking dictionaries
        # 1. Tracks how many hours each subject has been assigned per division.
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # {
        #   '1st_A': {'Math': 0, 'Physics': 0, ...},
        #   '2nd_B': {'OS': 0, 'DBMS': 0, ...},
        # }


        # 2. Tracks total teaching hours per division.
        division_total_hours = {
            f"{year}_{div}": 0
            for year in years
            for div in years[year]['divisions']
        }


        # {
        #   '1st_A': 0,
        #   '2nd_B': 0,
        # }


        # 3. Tracking Professors Workload
        # Tracks how many hours each professor teaches daily and weekly.
        prof_daily_hours = {prof: {day: 0 for day in days} for prof in self.all_professors}
        prof_total_hours = {prof: 0 for prof in self.all_professors}


        # prof_daily_hours
        # {
        #   'Prof_M1': {'Monday': 0, 'Tuesday': 0, ...}, #total hours on perticular day
        #   'Prof_VC': {'Monday': 0, 'Tuesday': 0, ...},
        # }

        # prof_total_hours
        # {
        #   'Prof_M1': 0,  # Total teaching hours in the week
        #   'Prof_VC': 0,
        # }

        # 4. Tracking Room Usage
        # Tracks how many times each classroom is used
        room_utilization = {room: 0 for room in classrooms}

        # {
        #   'N301': 0,  # Used 0 times in a week initially.
        #   'N302': 0,
        # }

        # 5. Tracking Section Workload
        # Tracks how many hours a section has classes per day.
        section_daily_hours = {div: {day: 0 for day in days} for div in self.all_divisions}

        # section_daily_hours
        # {
        #   '1st_A': {'Monday': 0, 'Tuesday': 0, ...},
        #   '2nd_B': {'Monday': 0, 'Tuesday': 0, ...},
        # }

        # 6. Tracks consecutive hours of teaching to avoid overloading students.
        section_consecutive_hours = {div: {day: [] for day in days} for div in self.all_divisions}

        # 1. Processing the chromosome: (5x6x16) (days x slots x rooms)
        for day_idx, day in enumerate(chromosome):
            for slot_idx, slot in enumerate(day):
                for room_idx, assignment in enumerate(slot):
                    if assignment:
                        div = assignment['division']
                        subj = assignment['subject']
                        prof = assignment['prof']
                        year = assignment['year']


                        # Assignment (Monday, 9-10 AM, N301):
                        # {
                        #   'division': '1st_A',
                        #   'subject': 'Math',
                        #   'prof': 'Prof_M1',
                        #   'year': '1st'
                        # }


                        # Track subject hours
                        division_subject_hours[div][subj] += 1 # increasing the count for that div with perticular subject
                        division_total_hours[div] += 1 # and total hours for division also increased.

                        # Track professor hours
                        prof_daily_hours[prof][days[day_idx]] += 1 # increasing professor working hour on perticular day.
                        prof_total_hours[prof] += 1 # increasing professor's total hours for week.

                        # Track room utilization
                        room_utilization[classrooms[room_idx]] += 1 # room utilization is increased by 1.

                        # Track section daily hours
                        # Tracks daily teaching load and consecutive teaching hours for each division.
                        section_daily_hours[div][days[day_idx]] += 1 # for perticular division what is the day workload.
                        section_consecutive_hours[div][days[day_idx]].append(slot_idx)


        # 2. Calculate penalties for each constraint
        #------------------------------ J
        # a) Each subject must be taught exactly 4 hours per division
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] != SUBJECT_HOURS_REQUIRED: # for perticular division and for perticular subject if subject hours not match to required subject hours(3) then it is not good solution.
                        penalty += 10000 * abs(division_subject_hours[full_div][subj] - SUBJECT_HOURS_REQUIRED) # variable penalty, if diff is large then penalty will large.

        # b) Each division must have correct total hours (24) cause 6 subjects and 4 hours in week
        for year in years:
            expected_hours = self.HOURS_PER_DIVISION[year]
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                if division_total_hours[full_div] != expected_hours: # for all division's total hours we re checking whether it is equal to expected hours if not large penalty.
                    penalty += 10000 * abs(division_total_hours[full_div] - expected_hours)

        #---------------------------- M
        # c) Professor daily workload balance
        # Goal: Ensure professors have a balanced workload across all days (no day is too overloaded or underloaded)

        # average daily hours per professor
        avg_prof_daily = sum(prof_total_hours.values()) / (len(self.all_professors) * len(days))
        # total teaching hours across all professors / average hours per professor per day
        for prof in self.all_professors: # for each professor
            for day in days: #on each day
                # compute the deviation from this average
                deviation = abs(prof_daily_hours[prof][day] - avg_prof_daily)
                # hours taught by the professor on that day - average daily hours per professor
                penalty += 1000 * deviation

        # d) Idle hours should be at end of day
        # Goal: Prefer schedules where empty slots are at the end of the day rather than between classes
        for day_idx, day in enumerate(chromosome): #for each day in the timetable(chromosome)
            last_used_slot = -1 # track the last used time slot (last_used_slot)
            for slot_idx in range(len(time_slots)): # for each time slot in the day
                # check whether any classroom in a given time slot is occupied
                slot_used = any(assignment is not None for assignment in day[slot_idx]) #True if at least one classroom in that time slot is occupied
                #                                    class-assignment in class wise list of assignments in specific time slot

                if slot_used:
                    last_used_slot = slot_idx # if the slot is used, update last_used_slot to the current slot index
                elif last_used_slot != -1:  # if found idle slot after used slot, give penalty
                    penalty += 50 * (slot_idx - last_used_slot)
                    # farther the idle slot is from the last used slot, higher the penalty

        # e) Avoid too many consecutive hours for professors
        # Prevent professors from teaching back-to-back for too long

        # INITIALIZE dictionary to track professor teaching hours
        prof_consecutive_hours = {} # empty dictionary
        for prof in self.all_professors: # for every professor
            prof_consecutive_hours[prof] = {} # sub-dictionary for each professor - store daily schedules
            for day in days: # for each day
                # initialize empty list to store time slots when professor teaches
                prof_consecutive_hours[prof][day] = []

        # COLLECT all time slots where each professor teaches
        for day_idx in range(len(chromosome)): #for each day
            day = chromosome[day_idx] #get schedule for current day
            for slot_idx in range(len(day)): #for each time slot in the day
                slot = day[slot_idx] #get all classroom assignments for current time slot
                for room_idx in range(len(slot)): #for each classroom in time slot
                    assignment = slot[room_idx] #get the assignment (class) scheduled in this classroom-time slot
                    if assignment is not None: #if classroom not empty
                        prof = assignment['prof'] #get professor teaching this class
                        day_name = days[day_idx] #get name of current day
                        prof_consecutive_hours[prof][day_name].append(slot_idx) #record the time slot index when the professor teaches on this day

        # check for CONSECUTIVE teaching hours
        for prof in self.all_professors: #loops through all professors again to analyze their schedules
            for day in days: # for each professor, checks each day's schedule
                slots = prof_consecutive_hours[prof][day] # get all time slots when the professor teaches on this day
                slots = sorted(slots) #sort chronologically
                consecutive_count = 1
                for i in range(1, len(slots)): #loops through the sorted time slots (starting from second)
                    if slots[i] - slots[i-1] == 1: # consecutive hours (if current slot is immediately after previous slot )
                        consecutive_count += 1
                        if consecutive_count > 2:  # prof teaches more than 2 consecutive hours
                            penalty += 1000
                    else:
                        consecutive_count = 1

        return -penalty  # return negative penalty (higher is better)
        # for us, higher penalty = worse timetable, lower penalty = better timetable
        # GA maximizes fitness functions so for GA, higher is better, which is not the case for us
        # hence return -penalty

    # ====================== INITIALIZATION ======================
    def initialize_population(self):
        #Create initial population of valid schedules
        self.population = []
        for _ in range(self.population_size):
            chromosome = self.create_valid_chromosome()
            self.population.append(chromosome)

    # ====================== SELECTION OPERATOR ======================
    #---- Tournament selection with elitism
    #takes fitness scores of the current population as input
    def selection(self, fitness_scores):
        selected = [] #list of chromosomes (timetables) that will be used for reproduction (crossover and mutation)
        # keep top 10% elites
        elite_size = max(1, int(self.population_size * 0.1)) #10% of the population size, min 1
        elite_indices = np.argsort(fitness_scores)[-elite_size:] # get indices of the top 10% fittest individuals
        selected.extend([self.population[i] for i in elite_indices]) #adds these elites directly to selected list for next generation

        # tournament selection for rest 90%
        for _ in range(self.population_size - elite_size): #90%
            candidates = random.sample(range(self.population_size), 5) # randomly select 5 candidates from the population
            best = max(candidates, key=lambda x: fitness_scores[x]) #select BEST individual with the highest fitness score
            selected.append(self.population[best]) # add the best individual to the selected list

        return selected #list of chromosomes (timetables) that will be used for reproduction (crossover and mutation)

    # ====================== REPRODUCTION OPERATORS ======================
    def crossover(self, parent1, parent2):
        #Enhanced crossover that preserves valid structures
        if random.random() > self.crossover_rate:
            return [row[:] for row in parent1], [row[:] for row in parent2]

        # Create empty children
        child1 = [[[None for _ in classrooms] for _ in time_slots] for _ in days]
        child2 = [[[None for _ in classrooms] for _ in time_slots] for _ in days]

        # Perform uniform crossover for each day
        for day_idx in range(len(days)):
            if random.random() < 0.5:  # Swap entire day
                child1[day_idx] = [slot[:] for slot in parent1[day_idx]]
                child2[day_idx] = [slot[:] for slot in parent2[day_idx]]
            else:
                child1[day_idx] = [slot[:] for slot in parent2[day_idx]]
                child2[day_idx] = [slot[:] for slot in parent1[day_idx]]

        return self.repair_schedule(child1), self.repair_schedule(child2)

    def mutate(self, chromosome):
        #mutation that maintains validity
        for _ in range(5):  # Perform 5 mutations
            # Find two random assignments to swap
            day1, day2 = random.sample(range(len(days)), 2)
            slot1, slot2 = random.sample(range(len(time_slots)), 2)
            room1, room2 = random.sample(range(len(classrooms)), 2)

            # Perform swap if both assignments exist
            if (chromosome[day1][slot1][room1] and
                chromosome[day2][slot2][room2]):
                # Check if swap would cause conflicts
                temp1 = chromosome[day1][slot1][room1].copy()
                temp2 = chromosome[day2][slot2][room2].copy()

                # Simulate swap
                chromosome[day1][slot1][room1] = None
                chromosome[day2][slot2][room2] = None

                # Check for conflicts
                conflict1 = self.has_conflict(chromosome, day1, slot1, temp2)
                conflict2 = self.has_conflict(chromosome, day2, slot2, temp1)

                if not conflict1 and not conflict2:
                    chromosome[day1][slot1][room1] = temp2
                    chromosome[day2][slot2][room2] = temp1
                else:
                    # Revert if conflict
                    chromosome[day1][slot1][room1] = temp1
                    chromosome[day2][slot2][room2] = temp2

        return self.repair_schedule(chromosome)

    # ====================== HELPER METHODS ======================
    def all_requirements_met(self, division_subject_hours, division_total_hours):
        for year in years:
            expected_hours = self.HOURS_PER_DIVISION[year]
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                # Check subject hours
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                        return False
                # Check total hours
                if division_total_hours[full_div] < expected_hours:
                    return False
        return True

    def select_remaining_assignment(self, division_subject_hours):
        remaining = []
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                for subj in years[year]['subjects']:
                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                        remaining.append((full_div, subj))
        return random.choice(remaining) if remaining else (None, None)

    def find_valid_timeslot(self, schedule, div, subj, year):
        # Try random slots first for diversity
        for _ in range(100):
            day_idx = random.randint(0, len(days)-1)
            slot_idx = random.randint(0, len(time_slots)-1)
            room_idx = random.randint(0, len(classrooms)-1)




             #check if room is empty --> if div is free
            if schedule[day_idx][slot_idx][room_idx] is None:
                # Check if division is available
                div_busy = any(
                    assignment and assignment['division'] == div
                    for assignment in schedule[day_idx][slot_idx]
                )
                if not div_busy:
                    # Check if professor is available
                    prof = random.choice(years[year]['professors'][subj])
                    prof_busy = any(
                        assignment and assignment['prof'] == prof
                        for assignment in schedule[day_idx][slot_idx]
                    )
                    if not prof_busy:
                        return day_idx, slot_idx, room_idx

        # If random search fails, do exhaustive search
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    if schedule[day_idx][slot_idx][room_idx] is None:
                        div_busy = any(
                            assignment and assignment['division'] == div
                            for assignment in schedule[day_idx][slot_idx]
                        )
                        if not div_busy:
                            prof = random.choice(years[year]['professors'][subj])
                            prof_busy = any(
                                assignment and assignment['prof'] == prof
                                for assignment in schedule[day_idx][slot_idx]
                            )
                            if not prof_busy:
                                return day_idx, slot_idx, room_idx
        return None, None, None

    def has_conflict(self, chromosome, day_idx, slot_idx, assignment):
        if not assignment:
            return False

        # Check division conflict
        for room_idx in range(len(classrooms)):
            existing = chromosome[day_idx][slot_idx][room_idx]
            if existing and existing['division'] == assignment['division']:
                return True

        # Check professor conflict
        for room_idx in range(len(classrooms)):
            existing = chromosome[day_idx][slot_idx][room_idx]
            if existing and existing['prof'] == assignment['prof']:
                return True

        return False

    def repair_schedule(self, chromosome):
          # Creates a dictionary to track how many hours each subject has been assigned per division.
        division_subject_hours = {
            f"{year}_{div}": {subj: 0 for subj in data['subjects']}
            for year, data in years.items()
            for div in data['divisions']
        }

        # First pass: count current assignments Iterates through the timetable (chromosome) and counts how many times each subject has been assigned for every division.
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    assignment = chromosome[day_idx][slot_idx][room_idx]
                    if assignment:
                        div = assignment['division']
                        subj = assignment['subject']
                        division_subject_hours[div][subj] += 1

        # Second pass: add missing assignments
        for day_idx in range(len(days)):
            for slot_idx in range(len(time_slots)):
                for room_idx in range(len(classrooms)):
                    if chromosome[day_idx][slot_idx][room_idx] is None:#loops through day,slot,room if it is empty then it is capable of getting any assignment which does not have any conflict

                          # Iterates over all divisions and subjects to find one that still needs more hours
                        for year in years:
                            for div in years[year]['divisions']:
                                full_div = f"{year}_{div}"
                                for subj in years[year]['subjects']:
                                    if division_subject_hours[full_div][subj] < SUBJECT_HOURS_REQUIRED:
                                        # Check if this slot is available
                                        test_assignment = {
                                            'division': full_div,
                                            'subject': subj,
                                            'prof': years[year]['professors'][subj][0],
                                            'year': year
                                        }
                                        if not self.has_conflict(chromosome, day_idx, slot_idx, test_assignment):
                                            prof = random.choice(years[year]['professors'][subj])
                                            chromosome[day_idx][slot_idx][room_idx] = {
                                                'division': full_div,
                                                'subject': subj,
                                                'prof': prof,
                                                'year': year
                                            }
                                            division_subject_hours[full_div][subj] += 1
                                            break
                                else:
                                    continue
                                break

        return chromosome

    # ====================== MAIN ALGORITHM ======================
    # ---------------J
    # This method runs the genetic algorithm (GA) to generate an optimal class timetable.
    # It follows the standard GA workflow:
    # 1. Initialize Population
    # 2. Evaluate Fitness
    # 3. Selection
    # 4. Crossover & Mutation
    # 5. Repeat for multiple generations
    # 6. Return the best timetable (chromosome)

    def run(self):
        # Creating an initial population (random valid timetables).
        # Each timetable (chromosome) is a 3D list representing [days][time_slots][rooms]
        self.initialize_population()

        # GA runs for a fixed number of generations (200)
        # Each generation evolves the timetable toward an optimal solution.
        for generation in range(self.generations):

            # Calculating Fitness for Each Chromosome (higher is better).
            # fitness_scores → Stores fitness for all schedules in the population.
            fitness_scores = [self.calculate_fitness(chromosome) for chromosome in self.population]

            # max_fitness is maximum fitness among all 100 individual's fitness for perticular generation.
            max_fitness = max(fitness_scores)

            # if max_fitness is greater than the best_fitness which is good then replace best_fitness with max_fitness.
            if max_fitness > self.best_fitness:
                self.best_fitness = max_fitness
                best_idx = fitness_scores.index(max_fitness) # maintaining the best_index.

                # Storing the best timetable found so far.
                self.best_chromosome = [
                    [
                        [
                            {k: v for k, v in assignment.items()} if assignment else None
                            for assignment in slot
                        ]
                        for slot in day
                    ]
                    for day in self.population[best_idx]
                ]

            # Print progress : Every 10 generations, prints: Best fitness, Average fitness of the population
            if generation % 10 == 0:
                avg_fitness = sum(fitness_scores) / len(fitness_scores)
                print(f"Gen {generation}: Best {self.best_fitness:.1f}, Avg {avg_fitness:.1f}")

            # Selection (Choosing Parents)
            selected = self.selection(fitness_scores) # Higher fitness increases selection probability.

            # Crossover and mutation
            new_population = []
            for i in range(0, len(selected), 2):
                parent1 = selected[i]
                parent2 = selected[i+1] if i+1 < len(selected) else selected[i]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.extend([self.mutate(child1), self.mutate(child2)])

            # Ensure population size
            new_population = new_population[:self.population_size]

            # Replace the worst individuals in new population with best from old population
            if self.best_chromosome:
                # Find worst in new population
                new_fitness = [self.calculate_fitness(chromosome) for chromosome in new_population]
                worst_idx = np.argmin(new_fitness)

                # Replace worst with best from previous generation
                new_population[worst_idx] = [
                    [
                        [
                            {k: v for k, v in assignment.items()} if assignment else None
                            for assignment in slot
                        ]
                        for slot in day
                    ]
                    for day in self.best_chromosome
                ]

            self.population = new_population

        return self.best_chromosome #The best timetable (least conflicts, most optimized) is returned.

    # ====================== OUTPUT METHODS ======================
    #------------J
    #  method is responsible for printing the generated timetable
    def print_schedule(self, chromosome):
        print("\n=== COMPLETE TIMETABLE (ALL DIVISIONS) ===") # Prints the full timetable for all divisions
        self.print_complete_schedule(chromosome)

        print("\n\n=== INDIVIDUAL DIVISION TIMETABLES ===") # Prints individual division-wise timetables
        self.print_division_schedules(chromosome)

        # Print statistics
        self.print_statistics(chromosome) # Prints statistics about the schedule.

    def print_complete_schedule(self, chromosome):

        for day_idx, day in enumerate(chromosome):
            print(f"\n{days[day_idx].upper()}") # Printing the day name (Monday, Tuesday, etc.) in uppercase.
            print(f"{'Time':<8}", end="") # Column for time slots

            #  Print room names as table headers
            for room in classrooms:
                print(f"{room:<30}", end="")
            print("\n" + "-"*180) # Table separator

            # Print time slots and class assignments
            for slot_idx in range(len(time_slots)):
                print(f"{time_slots[slot_idx]:<8}", end="") # Print time slot

                for room_idx in range(len(classrooms)):
                    assignment = day[slot_idx][room_idx] # Get class assignment for that time slot & room

                    if assignment and isinstance(assignment, dict): # Check if there is a valid class
                        output = (f"{assignment['year']}_{assignment['division']}-"
                                f"{assignment['subject'][:5]} ({assignment['prof']})")
                        print(f"{output:<30}", end="") # Print class info (year, division, subject, prof)
                    else:
                        print(f"{'---':<30}", end="") # Empty slot indicator
                print()

    def print_division_schedules(self, chromosome):
        for year in years:
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                print(f"\n\n=== TIMETABLE FOR {full_div} ===")

                # Create a division-specific timetable
                div_timetable = [[None for _ in time_slots] for _ in days]

                # Fill the division timetable
                for day_idx, day in enumerate(chromosome):
                    for slot_idx, slot in enumerate(day):
                        for room_idx, assignment in enumerate(slot):
                            if assignment and assignment['division'] == full_div:
                                div_timetable[day_idx][slot_idx] = {
                                    'subject': assignment['subject'],
                                    'prof': assignment['prof'],
                                    'room': classrooms[room_idx]
                                }

                # Print the division timetable
                for day_idx, day in enumerate(div_timetable):
                    print(f"\n{days[day_idx].upper()}")
                    print(f"{'Time':<8}{'Subject':<15}{'Professor':<20}{'Room':<10}")
                    print("-"*50)

                    for slot_idx in range(len(time_slots)):
                        assignment = day[slot_idx]
                        if assignment:
                            print(f"{time_slots[slot_idx]:<8}"
                                  f"{assignment['subject']:<15}"
                                  f"{assignment['prof']:<20}"
                                  f"{assignment['room']:<10}")
                        else:
                            print(f"{time_slots[slot_idx]:<8}{'---':<15}{'---':<20}{'---':<10}")

    def print_statistics(self, chromosome):
        print("\n=== STATISTICS ===")

        # Professor workload
        print("\nProfessor Workload:")
        prof_hours = defaultdict(int)
        for day in chromosome:
            for slot in day:
                for assignment in slot:
                    if assignment and isinstance(assignment, dict):
                        prof_hours[assignment['prof']] += 1

        avg_hours = sum(prof_hours.values()) / len(prof_hours) if prof_hours else 0
        print(f"Average hours per professor: {avg_hours:.1f}")
        for prof, hours in sorted(prof_hours.items()):
            print(f"{prof}: {hours} hours (deviation: {abs(hours-avg_hours):.1f})")

        # Division statistics by year
        print("\nDivision Coverage by Year:")
        for year in years:
            print(f"\n{year} Year:")
            for div in years[year]['divisions']:
                full_div = f"{year}_{div}"
                subj_hours = defaultdict(int)
                for day in chromosome:
                    for slot in day:
                        for assignment in slot:
                            if (assignment and isinstance(assignment, dict) and
                                assignment['division'] == full_div):
                                subj_hours[assignment['subject']] += 1

                total = sum(subj_hours.values())
                print(f"{div}: {total} total hours")
                for subj in years[year]['subjects']:
                    print(f"  {subj}: {subj_hours.get(subj, 0)}/{SUBJECT_HOURS_REQUIRED} hours")

# ====================== RUN THE ALGORITHM ======================
if __name__ == "__main__":
    # Create and run the GA
    print("Generating optimal timetable for 3 years with 4 divisions each...")
    ga = TimetableGA(population_size=300, generations=400)
    best_schedule = ga.run()

    # Print results
    ga.print_schedule(best_schedule)

    # Print final fitness
    print(f"\nFinal Fitness Score: {ga.best_fitness}")

Generating optimal timetable for 3 years with 4 divisions each...
Gen 0: Best -148872.7, Avg -179632.9
Gen 10: Best -146363.6, Avg -313336.3
Gen 20: Best -136381.8, Avg -155354.2
Gen 30: Best -131872.7, Avg -164818.3
Gen 40: Best -126381.8, Avg -149197.1
Gen 50: Best -120400.0, Avg -138658.3
Gen 60: Best -114400.0, Avg -120694.0
Gen 70: Best -109418.2, Avg -115855.4
Gen 80: Best -105927.3, Avg -120012.1
Gen 90: Best -102945.5, Avg -124838.3
Gen 100: Best -99454.5, Avg -102192.7
Gen 110: Best -95963.6, Avg -112869.6
Gen 120: Best -94472.7, Avg -107078.7
Gen 130: Best -92981.8, Avg -100386.3
Gen 140: Best -90981.8, Avg -95433.7
Gen 150: Best -87490.9, Avg -90319.4
Gen 160: Best -85490.9, Avg -87472.3
Gen 170: Best -83490.9, Avg -85633.0
Gen 180: Best -83490.9, Avg -84576.6
Gen 190: Best -82000.0, Avg -84297.8
Gen 200: Best -82000.0, Avg -83413.8
Gen 210: Best -82000.0, Avg -83008.6
Gen 220: Best -82000.0, Avg -82719.9
Gen 230: Best -82000.0, Avg -82680.1
Gen 240: Best -82000.0, Avg -8304